# Pose Detection with AlphaPhose

This notebook uses an open source project [MVIG-SJTU/AlphaPose](https://github.com/MVIG-SJTU/AlphaPose) to detect/track multi person poses on a given youtube video.

https://medium.com/the-owl/changing-the-python-version-on-google-colab-245fd510d3aeinstall-python-3-8-kernel-in-google-colaboratory/71511943#71511943

## Install Custom Python version and AlphaPose

In [ ]:
!apt-get update
!apt-get install -y build-essential libssl-dev libffi-dev libbz2-dev libncurses5-dev libncursesw5-dev zlib1g-dev libjpeg-dev libpng-dev

In [ ]:
# Crea ambiente virtuale con Python 3.9
!apt-get install python3.9 python3.9-distutils
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.9 1
!update-alternatives --config python3
!apt-get install python3-pip
!python3 -m pip install --upgrade pip --user

In [ ]:
! python3 -m pip install "numpy<2.0"
! python3 -m pip install matplotlib-inline
! python3 -m pip install visdom

In [ ]:
! python --version

In [ ]:
# ! pip3 install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu113
! pip3 install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
! pip install scipy
! pip install pillow
! pip install pyyaml
! pip install tqdm matplotlib nibabel pandas opencv-python gdown

In [ ]:
! sudo curl -L https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -o /usr/local/bin/yt-dlp
! sudo chmod a+rx /usr/local/bin/yt-dlp

In [ ]:
!python -m pip install cython
!sudo apt-get install libyaml-dev

In [ ]:
import os
! rm -rf /content/AlphaPose
os.chdir('/content/')

In [ ]:
# ! git clone https://github.com/MVIG-SJTU/AlphaPose.git
# os.chdir('/content/AlphaPose')
# print(os.getcwd())
# ! python setup.py build develop

In [ ]:
import os
from os.path import exists, join, basename, splitext

import time
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

from IPython.display import YouTubeVideo

git_repo_url = 'https://github.com/MVIG-SJTU/AlphaPose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install dependencies
  !git clone -q -b pytorch --depth 1 $git_repo_url
  # !cd $project_name && pip install -q -r requirements.txt

import sys
sys.path.append(project_name)

## Download pretrained models

In [ ]:
import gdown
import requests

# Replace 'your-file-id' with the actual file ID from Google Drive
file_id = '1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW'
pretrained_model_path = join(project_name, 'models/sppe/duc_se.pth')
gdown.download(f"https://drive.google.com/uc?id={file_id}", pretrained_model_path, quiet=False)

# yolo_file_id = '1D47msNOOiJKvPOXlnpyzdKA3k6E97NTC'
yolo_pretrained_model_path = join(project_name, 'models/yolo/yolov3-spp.weights')
# gdown.download(f"https://drive.google.com/uc?id={file_id}", yolo_pretrained_model_path, quiet=False)

# Make a GET request to download the file
response = requests.get("https://pjreddie.com/media/files/yolov3-spp.weights", stream=True)

# Save the file to your local system
with open(yolo_pretrained_model_path, 'wb') as file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            file.write(chunk)

In [ ]:
## fix code for AlphaPose

with open("/content/AlphaPose/fn.py", "r") as f:
    python_file = "".join(f.readlines())

python_file = python_file.replace("from torch._six import string_classes, int_classes", "from torch._six import string_classes\nint_classes = int")

with open("/content/AlphaPose/fn.py", "w") as f:
    f.write(python_file)

## Detect poses on a test video

We are going to detect poses on the following youtube video:

In [ ]:
YOUTUBE_ID = '0vnMdFmZ3P8'

YouTubeVideo(YOUTUBE_ID)

Download the above youtube video, cut the first 5 seconds and do the pose detection on that 5 seconds:

In [ ]:
# ! rm -df youtube.mp4
# download the youtube with the given ID
! yt-dlp --output "youtube.mp4" https://www.youtube.com/watch?v=$YOUTUBE_ID
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -t 5 video.mp4

In [ ]:
# run AlphaPose_video on these 5 seconds video
# !rm -rf AlphaPose_video.avi

!cd AlphaPose && python3 video_demo.py --sp --video ../video.mp4 --outdir .. --save_video --vis_fast
#! python3 AlphaPose/video_demo.py --sp --video video.mp4 --outdir . --save_video

# convert the result into MP4
!ffmpeg -y -loglevel info -i AlphaPose_video.avi AlphaPose_video.mp4

Finally, visualize the result:

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('AlphaPose_video.mp4', width=960, height=720)